In [ ]:
# Batchsizes, epoch number, training data number are all reduced for displaying purposes # Importing the required modules
from tqdm.notebook import tqdm
import time
import os
import json
import requests
from struct import unpack
import h5py
import pickle
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt import tensorflow as tf
import random
import PIL
from PIL import Image
import keras
from keras import Input, Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, \
concatenate, Flatten, Lambda, Concatenate, add, Input, LSTM, GRU, RNN, Embedding, Multiply, TimeDistributed, Bidirectional, RepeatVector, Activation, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from keras_preprocessing.sequence import pad_sequences from keras.utils import to_categorical
from keras.models import Model, Sequential
# path names can be changed for downloading the data somewhere else
datTrainFileName = "eee443_project_dataset_train.h5"

 imTrainDirect = "train_images/" feaTrainDirect = "train_tupled_data"
datTestFileName = "eee443_project_dataset_test.h5" imTestDirect = "test_images/"
feaTestDirect = "test_tupled_data"
# Functions to obtain the data
def imgDown(data, namPath):
# In this function, we downloaded the images from the given url
tim = time.time() i=0
if not os.path.exists(namPath): os.makedirs(namPath)
for url in data:
url = url.decode()
name = url.split("/")[-1].strip()
path = os.path.join(namPath, name)
if not os.path.exists(path):
hed = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
resp = requests.get(url, stream=True)
# check if data is obttained successfully if resp.status_code == 200:
with open(path, 'wb') as outfile: outfile.write(resp.content)
# prints affirmation at each 1000 iterations if i % 1000 == 1:
timRes = time.time() - tim
timPerIter = timRes/i
print("Total passed minute is: {:.2f} . There is {:.2f} seconds per iteration. Our iteration
is: {}".format(timRes/60, timPerIter, i)) i += 1
# Functions for data processing

 def StringFromCaptArry(captArry):
# This function is used for prediction and demonstration
captList = []
capt = ""
if(captArry.ndim == 1):
captArry = np.expand_dims(captArry, axis=0)
for caps in captArry:
for NamWord in caps:
if (NamWord == 'x_NULL_') or (NamWord == 'x_START_') or (NamWord == 'x_END_'):
continue
capt += NamWord + " "
captList.append(capt.strip()) capt = ""
return captList
def captGet(namList, imid, cap, name): indexx = namList.index(name) + imid.min() return cap[np.where(imid == indexx)]
def PreProcCreation(directImg, shuffle=False):
def fileMethod(path):
img = tf.io.read_file(path)
img = tf.image.decode_jpeg(img, channels=3) img = tf.image.resize(img, (64, 64))
return img
def nameProcess(path):
name = path.numpy().decode().split("\\")[-1]

 return name
def proc(path):
name = tf.py_function(nameProcess, [path], tf.string) img = tf.py_function(fileMethod, [path], tf.float32) return (img, name)
datFile = tf.data.Dataset.list_files(str(directImg) + "*.jpg", shuffle=shuffle) return datFile.map(lambda x: proc(x))
def create_features(namFile, ims, namList, imid, cap, process_size = 250):
length = 0
with open(namFile, "wb") as outfile:
for data in tqdm(ims.batch(process_size)):
image = data[0]
name = data[1].numpy()
feature = image
for i in range(feature.shape[0]):
feat = feature[i]
namm = name[i].decode()
cpt = captGet(namList, imid, cap, namm)
tp = (feat, cpt, namm) pickle.dump(tp, outfile)
length += 1
outfile.close() return length
def installPickle(namFile):
with open(namFile, "rb") as f: while True:
try:
yield pickle.load(f)

 except EOFError: break
def dataGet(directFeat, url=None, imid=None, cap=None, directImg=None):
# Main function that combines the captions and corresponding images
len = -1
if not os.path.isfile(directFeat):
if not directImg:
raise Exception("No image directory given. Enter image directory for feature
extraction.")
listName = [u.split("/")[-1].strip() for u in np.char.decode(url).tolist()] images = PreProcCreation(directImg)
len = create_features(directFeat, images, listName, imid, cap)
dset = tf.data.Dataset.from_generator(installPickle, args=[directFeat], output_types=(np.float32,np.int32, tf.string))
if len == -1:
len = dset.reduce(0, lambda x, _: x + 1).numpy()
return dset, len
def GetEmbed(w2ix):
# This function creates the embedding matrix with our vocabulary
# Load Glove vectors pathGlove = 'glove.6B.200d.txt'
indxEmbed = {} # empty dictionary
fil = open(pathGlove, encoding="utf-8")
for line in fil:
val = line.split()


 word = val[0]
coefficients = np.asarray(val[1:], dtype='float32') # if (word == 'startseq' or word == 'unk' ):
# print(word)
indxEmbed[word] = coefficients fil.close()
print('Our total vector of word is: %s' % len(indxEmbed))
dimEmbed = 200 totVocab = 1004
# Get 200-dim dense vector for each of the 10000 words in out vocabulary mtxEmbed = np.zeros((totVocab, dimEmbed))
for word, i in w2ix.items():
if (word == 'x_UNK_'): word = 'unk'
vectEmbed = indxEmbed.get(word) if vectEmbed is None:
print('There is no existing word for capturing')
if vectEmbed is not None:
# Words not found in the embedding index will be all zeros mtxEmbed[i] = vectEmbed
return mtxEmbed
def HotOne(X, size=1004):
X = X -1
onehott = np.zeros((X.shape[0], 0)) temp = np.zeros((X.shape[0], size)) tempp = np.arange(X.shape[0]) temp[tempp, X[:,0]] = 1
onehott = np.hstack((onehott, temp))
return onehott
# Get the train data
fill = h5py.File(datTrainFileName, 'r') for key in list(fill.keys()):

 print(key, ":", fill[key][()].shape)
captTrain = fill["train_cap"][()] imidTrain = fill["train_imid"][()] urlTrain = fill["train_url"][()] codeWord = fill["word_code"][()]
dif = pd.DataFrame(codeWord) dif = dif.sort_values(0, axis=1) words = np.asarray(dif.columns)
w2ixx = {}
for i in range(len(words)):
word = words[i] w2ixx[word] = i
# To download the images, this code needs to be run but lasts long
# If the data is already downloaded, path name can be changed accordingly.
# save_ims(train_url,TRAIN_IMAGES_DIRECTORY )
# This part is used for combining the images and their related captions using # the train_imid data, also the image resolutions are fixed to 64x64
datTrain, datTrainLen = dataGet( feaTrainDirect, urlTrain, imidTrain, captTrain, imTrainDirect)
print( "Total data obtaining proportion is: {} / {} ".format(datTrainLen, len(urlTrain)) )
# Similar with the training data, test data is combined and resolutions are fixed
fill = h5py.File(datTestFileName, "r")
captTest = fill["test_caps"][()] imidTest = fill["test_imid"][()] urlTest = fill["test_url"][()]
datTest, datTestLength = dataGet(feaTestDirect, urlTest, imidTest, captTest, imTestDirect) print( "The proportion of data obtaining is: {} / {} ".format(datTestLength, len(urlTest), ))
for d in datTrain.shuffle(500).take(1): feats = d[0]
namImg= d[2].numpy().decode()

 capt = d[1].numpy()
imgg = cv2.imread(imTrainDirect + namImg)
imgg = cv2.cvtColor(imgg, cv2.COLOR_BGR2RGB) plt.imshow(imgg)
plt.show()
captt = StringFromCaptArry(words[capt])
print(captt[0])
print(feats.shape, capt.shape)
for d in datTrain.shuffle(1000).take(1): feats = d[0].numpy()
namImg= d[2].numpy().decode() capt = d[1].numpy()
xxx = feats plt.imshow(xxx.astype('uint8'))
# Creating the embedding matrix from glove vectors
matrixEmbed = GetEmbed(w2ixx)
# Functions for the GAN architecture
def lossKL(corY, predY):
u = predY[:, :128]
sigLogarithm = predY[:, :128]
totLoss = -sigLogarithm + .5 * (-1 + K.exp(2. * sigLogarithm) + K.square(u)) totLoss = K.mean(totLoss)
return totLoss
def LossCustGen(corY, predY):
return K.binary_crossentropy(corY, predY)
def ImgSave(img, path):
#rgb image saving
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1) ax.imshow(img) ax.axis("off") ax.set_title("Image")

 plt.savefig(path) plt.close()
def cGen(x):
u = x[:, :128]
sigmaLogaritmm = x[:, 128:]
derivStandart = K.exp(sigmaLogaritmm)
epsilon = K.random_normal(shape=K.constant((u.shape[1],), dtype='int32')) crr = derivStandart * epsilon + u
return crr
def caModelBuild(): # Cond
totLayerInp = Input(shape=(3400,))
x = Dense(256)(totLayerInp)
x = LeakyReLU(alpha=0.2)(x)
mod = Model(inputs=[totLayerInp], outputs=[x]) return mod
def CompresBuildEmbedModel():
# Compressing the embedding matrix
totLayerInp = Input(shape=(3400,)) x = Dense(128)(totLayerInp)
x = ReLU()(x)
mod = Model(inputs=[totLayerInp], outputs=[x]) return mod
def stage1BuildGen():
totLayerInp = Input(shape=(3400,))
dns = Dense(256)(totLayerInp) sigmaLogU = LeakyReLU(alpha=0.2)(dns)
crr = Lambda(cGen)(sigmaLogU)
totLayerInp2 = Input(shape=(100,))
InpGen = Concatenate(axis=1)([crr, totLayerInp2])
dns = Dense(128 * 8 * 4 * 4, use_bias=False)(InpGen) dns = ReLU()(dns)
dns = Reshape((4, 4, 128 * 8), input_shape=(128 * 8 * 4 * 4,))(dns)
dns = UpSampling2D(size=(2, 2))(dns)
dns = Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(dns) dns = BatchNormalization()(dns)
dns = ReLU()(dns)
dns = UpSampling2D(size=(2, 2))(dns)
dns = Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(dns) dns = BatchNormalization()(dns)
dns = ReLU()(dns)
dns = UpSampling2D(size=(2, 2))(dns)
dns = Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(dns) dns = BatchNormalization()(dns)
dns = ReLU()(dns)
dns = UpSampling2D(size=(2, 2))(dns)
dns = Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(dns) dns = BatchNormalization()(dns)
dns = ReLU()(dns)
dns = Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(dns) dns = Activation(activation='tanh')(dns)
wrkStage1 = Model(inputs=[totLayerInp, totLayerInp2], outputs=[dns,sigmaLogU])
return wrkStage1 def stage1BuildDisc():
totLayerInp = Input(shape=(64, 64, 3))
dns = Conv2D(64, (4, 4), padding='same', strides=2,
input_shape=(64, 64, 3), use_bias=False)(totLayerInp) dns = LeakyReLU(alpha=0.2)(dns)
dns = Conv2D(128, (4, 4), padding='same', strides=2, use_bias=False)(dns) dns = BatchNormalization()(dns)
dns = LeakyReLU(alpha=0.2)(dns)
dns = Conv2D(256, (4, 4), padding='same', strides=2, use_bias=False)(dns) dns = BatchNormalization()(dns)
dns = LeakyReLU(alpha=0.2)(dns)
dns = Conv2D(512, (4, 4), padding='same', strides=2, use_bias=False)(dns) dns = BatchNormalization()(dns)


 dns = LeakyReLU(alpha=0.2)(dns) totLayerInp2 = Input(shape=(4, 4, 128)) InpComb = concatenate([dns, totLayerInp2])
dns2 = Conv2D(64 * 8, kernel_size=1, padding="same", strides=1)(InpComb)
dns2 = BatchNormalization()(dns2) dns2 = LeakyReLU(alpha=0.2)(dns2) dns2 = Flatten()(dns2)
dns2 = Dense(1)(dns2)
dns2 = Activation('sigmoid')(dns2)
modDisStage1 = Model(inputs=[totLayerInp, totLayerInp2], outputs=[dns2]) return modDisStage1
def advModelBuild(ModGen, modDis): totLayerInp = Input(shape=(3400,)) totLayerInp2 = Input(shape=(100,)) totLayerInp3 = Input(shape=(4, 4, 128))
dns, sigmaLogU = ModGen([totLayerInp, totLayerInp2])
modDis.trainable = False
corr = modDis([dns, totLayerInp3])
modd = Model(inputs=[totLayerInp, totLayerInp2, totLayerInp3], outputs=[corr, sigmaLogU])
return modd
lenData = 30 #train_data_length trData = datTrain.take(lenData)
lenValue = round(lenData * 0.15) trValue = lenData - lenValue
datVal = trData.take(lenValue) datTr = trData.skip(lenValue)
# Captions and Images are received from train and val data
str = []
strr = []
for data in datTr:


 feat = data[0].numpy() cp = data[1].numpy()
str.append(feat) strr.append(cp[0])
sttr = []
sttrr = []
for data in datVal:
featt = data[0].numpy() cpp = data[1].numpy()
sttr.append(featt) sttrr.append(cpp[0])
# Here, text embedding is done using the embedding matrix and it is flattened.
relEmbed = []
for i in range(trValue):
emb = np.array(strr[i:i+1])
em = np.transpose(emb)
emm = HotOne(em)
emmm = np.matmul(emm,matrixEmbed) relEmbed.append(emmm)
bssEm = np.array(relEmbed)
bssEm = bssEm.reshape((trValue,3400)) bssEm.shape
datNum1 = lenValue relEmb1 = []
for i in range(lenValue):
emb1 = np.array(sttrr[i:i+1])
em1 = np.transpose(emb1)
emm1 = HotOne(em1)
emmm1 = np.matmul(emm1,matrixEmbed) relEmb1.append(emmm1)
bssEm1 = np.array(relEmb1)
bssEm1 = bssEm1.reshape((lenValue,3400))
bssEm1.shape,bssEm.shape
trAr = np.array(str[0:trValue]) trEmbed = bssEm

 valAr = np.array(sttr[0:trValue]) valEmbed = bssEm1
trAr.shape,valAr.shape
# The training of the model
if __name__ == '__main__':
totImg = 64 totBatch = 4 dimensionZ = 100 LRgenST1 = 0.001 LRdistST1 = 0.001 LRdsST1 = 600 epochs = 1000 totCond = 128
# Optimizers that we use for generator and discriminator Optdis = Adam(lr=LRdistST1, beta_1=0.5, beta_2=0.999) Optgen = Adam(lr=LRgenST1, beta_1=0.5, beta_2=0.999)
# Networks implemented and build
ModCA = caModelBuild() ModCA.compile(loss="binary_crossentropy", optimizer="adam")
disST1 = stage1BuildDisc() disST1.compile(loss='binary_crossentropy', optimizer=Optdis)
genST1 = stage1BuildGen() genST1.compile(loss="mse", optimizer=Optgen)
modEmbedComp = CompresBuildEmbedModel() modEmbedComp.compile(loss="binary_crossentropy", optimizer="adam")
modAdver = advModelBuild(ModGen=genST1, modDis=disST1) modAdver.compile(loss=['binary_crossentropy', lossKL], loss_weights=[1, 2.0],
optimizer=Optgen, metrics=None)
# Fake and Real Labels
labCor = np.ones((totBatch, 1), dtype=float) * 0.9

 labWro = np.zeros((totBatch, 1), dtype=float)
for epoch in range(epochs):
print("Please wait...")
print("Total Epoch:", epoch)
print("Total Batches:", int(trAr.shape[0] / totBatch))
LossGen = [] LossDis = []
# Here is batch gradient descend totBatchh = int(trAr.shape[0] / totBatch) for index in range(totBatchh):
print("Total Batch:{}".format(index+1))
# Batch data and noise,
noiseZ = np.random.normal(0, 1, size=(totBatch, dimensionZ)) batchImg = trAr[index * totBatch:(index + 1) * totBatch] batchEmbed = trEmbed[index * totBatch:(index + 1) * totBatch] batchImg = (batchImg - 127.5) / 127.5
# Forward pass through the generator network
WroImg, _ = genST1.predict([batchEmbed, noiseZ], verbose=3)
# Compressing the text embedding further
embedComp = modEmbedComp.predict_on_batch(batchEmbed) embedComp = np.reshape(embedComp, (-1, 1, 1, totCond)) embedComp = np.tile(embedComp, (1, 4, 4, 1))
RealLossDis = disST1.train_on_batch([batchImg, embedComp], np.reshape(labCor, (totBatch, 1)))
FakeLossDis = disST1.train_on_batch([WroImg, embedComp], np.reshape(labWro, (totBatch, 1)))
WrongLossDis = disST1.train_on_batch([batchImg[:(totBatch - 1)], embedComp[1:]], np.reshape(labWro[1:], (totBatch-1, 1)))
LossD = 0.5 * np.add(RealLossDis, 0.5 * np.add(WrongLossDis, FakeLossDis))
# Training of the generator network
LossG = modAdver.train_on_batch([batchEmbed, noiseZ, embedComp],[K.ones((totBatch, 1)) * 0.9, K.ones((totBatch, 256)) * 0.9])
LossDis.append(LossD) LossGen.append(LossG)

 print("Loss of D:{}".format(LossD)) print("Loss of G:{}".format(LossG))
if epoch % 1 == 0:
# Predict on Validation see the loss
NoiseZZ = np.random.normal(0, 1, size=(totBatch, dimensionZ)) batchEmbed = valEmbed[0:totBatch]
WroImg, _ = genST1.predict_on_batch([batchEmbed, NoiseZZ]) lossVal = modAdver.evaluate([batchEmbed, noiseZ,
embedComp],[K.ones((totBatch, 1)) * 0.9, K.ones((totBatch, 256)) * 0.9]) print("Validation Loss:{}".format(lossVal))
# Save images
for i, img in enumerate(WroImg[:10]):
ImgSave(img, "Printed Results/gen_{}_{}.png".format(epoch, i))
# Save the model in each epoch
# stage1_gen.save_weights("stage1_gen.h5") # stage1_dis.save_weights("stage1_dis.h5")
# summary of generator network genST1.summary()
lenTest = 3000 #test_data_length datTest = datTest.take(lenTest)
xRowColll = [] xRowwColll = [] for data in datTest:
feattt = data[0].numpy() cpsss = data[1].numpy()
xRowColll.append(feattt) xRowwColll.append(cpsss[0])
realEmbbb = []
for i in range(lenTest):
embeddd = np.array(xRowwColll[i:i+1])
emRowColll = np.transpose(embeddd)
emRowwColll = HotOne(emRowColll)
emRowwwColll = np.matmul(emRowwColll,matrixEmbed) realEmbbb.append(emRowwwColll)
bassEmbeddd = np.array(realEmbbb)

bassEmbeddd = bassEmbeddd.reshape((lenTest,3400)) bassEmbeddd.shape
testX = np.array(str[0:lenTest]) testEmbeds = bssEm
testX.shape,testEmbeds.shape
btcEmbed1 = testEmbeds[0:totBatch]
imgWro = genST1.predict_on_batch([btcEmbed1, NoiseZZ])
cpExamp = np.array(strr)
cpp = StringFromCaptArry(words[strr[0]])
print(cpp)
# needs to be trained, output won't give a good result imggg = imgWro[0]
plt.imshow(imggg[0])